<a href="https://colab.research.google.com/github/jewoolee0502/ComputerVision/blob/main/ECSE415_A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE 415: Introduction to Computer Vision
###### Jewoo Lee - 260910789
###### Anthony Bonta - 261053688

## Assignment 5: Video Analysis

### Library Requirements

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Installs
!pip install -q ultralytics
!pip install -q kaggle

# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import math
import torch

from ultralytics import YOLO

#### Path

In [ ]:
path = '/content/drive/MyDrive/McGill/2025/Fall 2025/ECSE 415/A5/'

### 1. Data Preparation

### 2. Model Implementation

### 3. Model Evaluation

### 4. Prediction & Kaggle Competition